# Pulse Arithmetic Lab - Interactive Notebook

**No hardware required.** This notebook demonstrates the algorithms used in Pulse Arithmetic Lab using pure Python/NumPy.

The ESP32-C6 firmware implements these same algorithms in fixed-point arithmetic on real hardware.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EntroMorphic/pulse-arithmetic-lab/blob/main/notebooks/pulse_arithmetic_lab.ipynb)

---

## Contents

1. **Pulse Counting = Addition** - The simplest computation
2. **Ternary Dot Products** - Neural network inference without multiplication
3. **Spectral Oscillators** - Complex-valued dynamics
4. **Equilibrium Propagation** - Learning without backpropagation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

# Set up nice plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
np.random.seed(42)

---

## 1. Pulse Counting = Addition

The fundamental insight: a counter that increments by 1 for each pulse performs **addition in hardware**.

```
Initial: count = 0
Pulse 1: count = 1
Pulse 2: count = 2
...
Pulse N: count = N
```

The ESP32-C6's PCNT peripheral does this without CPU involvement.

In [ ]:
def pulse_counter(num_pulses: int) -> int:
    """Simulate PCNT peripheral: count pulses."""
    count = 0
    for _ in range(num_pulses):
        count += 1  # Each pulse increments counter
    return count

# Test it
for n in [10, 100, 1000, 10000]:
    result = pulse_counter(n)
    status = "PASS" if result == n else "FAIL"
    print(f"Pulses: {n:5d} -> Count: {result:5d} [{status}]")

In [ ]:
# Addition via sequential pulses
def add_via_pulses(a: int, b: int) -> int:
    """Add two numbers by counting pulses."""
    count = 0
    for _ in range(a):
        count += 1
    for _ in range(b):
        count += 1
    return count

# Test
print("\nAddition via pulses:")
for a, b in [(5, 3), (100, 50), (1000, 2000)]:
    result = add_via_pulses(a, b)
    expected = a + b
    status = "PASS" if result == expected else "FAIL"
    print(f"  {a} + {b} = {result} (expected {expected}) [{status}]")

---

## 2. Ternary Dot Products

Standard neural network: `y = sum(w_i * x_i)` requires N multiplications.

**Ternary weights {-1, 0, +1}** eliminate multiplication:
- `w = +1`: add x to accumulator
- `w = 0`: skip
- `w = -1`: subtract x from accumulator

With PCNT, we route +1 weights to increment channel, -1 weights to decrement channel.

In [ ]:
def ternary_dot_product(x: np.ndarray, w: np.ndarray) -> int:
    """
    Dot product with ternary weights.
    
    Args:
        x: Input vector
        w: Weight vector with values in {-1, 0, +1}
    """
    assert np.all(np.isin(w, [-1, 0, 1])), "Weights must be ternary"
    
    accumulator = 0
    for i in range(len(x)):
        if w[i] == 1:
            accumulator += x[i]
        elif w[i] == -1:
            accumulator -= x[i]
        # w[i] == 0: do nothing
    
    return accumulator

# Compare with standard dot product
x = np.array([1, 2, 3, 4])
w = np.array([1, -1, 1, -1])  # Ternary weights

ternary_result = ternary_dot_product(x, w)
standard_result = np.dot(x, w)

print(f"Input x:         {x}")
print(f"Ternary weights: {w}")
print(f"Ternary dot:     {ternary_result}")
print(f"Standard dot:    {standard_result}")
print(f"Match: {ternary_result == standard_result}")

In [ ]:
# Simulate 4 parallel neurons with ternary weights
def parallel_dot_products(x: np.ndarray, weight_matrix: np.ndarray) -> np.ndarray:
    """
    Compute multiple dot products in parallel.
    
    In hardware: PARLIO sends bits to 4 PCNT units simultaneously.
    """
    num_neurons = weight_matrix.shape[0]
    outputs = np.zeros(num_neurons, dtype=int)
    
    for n in range(num_neurons):
        outputs[n] = ternary_dot_product(x, weight_matrix[n])
    
    return outputs

# Define weight patterns (matching firmware)
weights = np.array([
    [+1, +1, +1, +1],  # Neuron 0: sum all
    [-1, -1, -1, -1],  # Neuron 1: negate all
    [+1, -1, +1, -1],  # Neuron 2: alternating
    [+1, +1, -1, -1],  # Neuron 3: first half + second half -
])

# Test with various inputs
test_inputs = [
    np.array([1, 1, 1, 1]),
    np.array([10, 10, 10, 10]),
    np.array([1, 2, 3, 4]),
]

print("Parallel Dot Products:")
print("="*50)
for x in test_inputs:
    outputs = parallel_dot_products(x, weights)
    print(f"\nInput: {x}")
    for n, (w, o) in enumerate(zip(weights, outputs)):
        print(f"  Neuron {n} (w={w}): output = {o}")

---

## 3. Spectral Oscillators

Real-valued neurons have magnitude but no phase. **Complex-valued oscillators** have both:

$$z = r \cdot e^{i\theta} = r\cos(\theta) + ir\sin(\theta)$$

Key properties:
- **Rotation**: Phase advances over time
- **Decay**: Magnitude decreases (band-specific rates)
- **Coherence**: How aligned are the phases?

In [ ]:
class SpectralOscillator:
    """
    Complex-valued oscillator network with band-specific dynamics.
    """
    
    BAND_NAMES = ["Delta", "Theta", "Alpha", "Gamma"]
    BAND_DECAY = [0.98, 0.90, 0.70, 0.30]  # Slow to fast decay
    BAND_FREQ = [0.1, 0.3, 1.0, 3.0]       # Slow to fast rotation
    
    def __init__(self, neurons_per_band=4):
        self.num_bands = 4
        self.neurons_per_band = neurons_per_band
        
        # Initialize with random phases, unit magnitude
        self.phases = np.random.uniform(0, 2*np.pi, 
                                        (self.num_bands, neurons_per_band))
        self.magnitudes = np.ones((self.num_bands, neurons_per_band)) * 0.9
        
    def evolve(self, steps=1, input_energy=None):
        """Evolve the oscillators for given steps."""
        history = {'magnitudes': [], 'phases': [], 'coherence': []}
        
        for _ in range(steps):
            # Inject input energy
            if input_energy is not None:
                mask = self.magnitudes < 0.5
                self.magnitudes[mask] += 0.1 * input_energy
            
            # Rotate and decay each band
            for b in range(self.num_bands):
                self.phases[b] += self.BAND_FREQ[b] * 0.1
                self.magnitudes[b] *= self.BAND_DECAY[b]
            
            # Wrap phases to [0, 2pi)
            self.phases = self.phases % (2 * np.pi)
            
            # Record history
            history['magnitudes'].append(self.magnitudes.copy())
            history['phases'].append(self.phases.copy())
            history['coherence'].append(self.get_coherence())
        
        return history
    
    def get_coherence(self, band=None):
        """
        Kuramoto order parameter: coherence = |mean(e^(i*theta))|
        """
        if band is not None:
            phases = self.phases[band]
            mags = self.magnitudes[band]
        else:
            phases = self.phases.flatten()
            mags = self.magnitudes.flatten()
        
        # Only count oscillators with meaningful magnitude
        valid = mags > 0.01
        if not np.any(valid):
            return 0.0
        
        # Unit vectors
        z = np.exp(1j * phases[valid])
        return np.abs(np.mean(z))

In [ ]:
# Demonstrate band-specific decay
osc = SpectralOscillator()

# Inject energy and evolve
osc.evolve(steps=10, input_energy=0.5)
history = osc.evolve(steps=100, input_energy=None)

# Plot magnitude decay
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Magnitude over time
ax = axes[0]
mags = np.array(history['magnitudes'])
for b, name in enumerate(SpectralOscillator.BAND_NAMES):
    ax.plot(mags[:, b, 0], label=name, linewidth=2)
ax.set_xlabel('Time Step')
ax.set_ylabel('Magnitude')
ax.set_title('Band-Specific Decay Rates')
ax.legend()
ax.grid(True, alpha=0.3)

# Coherence over time
ax = axes[1]
ax.plot(history['coherence'], 'k-', linewidth=2)
ax.set_xlabel('Time Step')
ax.set_ylabel('Coherence')
ax.set_title('Global Phase Coherence')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nFinal state:")
for b, name in enumerate(SpectralOscillator.BAND_NAMES):
    avg_mag = np.mean(osc.magnitudes[b])
    coh = osc.get_coherence(b)
    print(f"  {name}: magnitude={avg_mag:.4f}, coherence={coh:.4f}")

---

## 4. Equilibrium Propagation

**The problem with backpropagation:**
- Requires separate forward and backward passes
- Backward pass needs non-local information
- Not biologically plausible

**Equilibrium propagation solution:**
1. **Free phase**: Let system evolve to equilibrium
2. **Nudged phase**: Clamp output toward target, evolve again
3. **Update**: `Δw = η * (correlation_nudged - correlation_free)`

The gradient emerges from the difference between two forward passes!

In [ ]:
class EquilibriumPropNetwork:
    """
    Simple network that learns via equilibrium propagation.
    """
    
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=4):
        self.W_in = np.random.randn(hidden_dim, input_dim) * 0.1
        self.W_out = np.random.randn(output_dim, hidden_dim) * 0.1
        
        self.hidden = np.zeros(hidden_dim)
        self.output = np.zeros(output_dim)
    
    def forward(self, x, steps=30):
        """Run to equilibrium."""
        self.hidden = np.tanh(self.W_in @ x)
        for _ in range(steps):
            self.output = np.tanh(self.W_out @ self.hidden)
        return self.output
    
    def nudged_forward(self, x, target, steps=30, beta=0.5):
        """Run with output nudged toward target."""
        self.hidden = np.tanh(self.W_in @ x)
        for _ in range(steps):
            raw = self.W_out @ self.hidden
            nudged = raw + beta * (target - np.tanh(raw))
            self.output = np.tanh(nudged)
        return self.output
    
    def train_step(self, x, target, lr=0.01, beta=0.5):
        """One equilibrium propagation update."""
        # Free phase
        self.forward(x)
        free_output = self.output.copy()
        corr_out_free = np.outer(self.output, self.hidden)
        
        # Nudged phase
        self.nudged_forward(x, target, beta=beta)
        corr_out_nudged = np.outer(self.output, self.hidden)
        
        # Update weights
        self.W_out += lr * (corr_out_nudged - corr_out_free)
        
        # Return loss
        return np.mean((free_output - target)**2)

In [ ]:
# Train on 2-pattern discrimination task
net = EquilibriumPropNetwork(input_dim=4, hidden_dim=16, output_dim=4)

# Training patterns
patterns = [
    (np.array([0, 0, 1, 1]), np.array([1, 0, 0, 0])),   # Pattern 0
    (np.array([1, 1, 0, 0]), np.array([0, 0, 0, 1])),   # Pattern 1
]

# Training loop
losses = []
separations = []

for epoch in range(200):
    epoch_loss = 0
    outputs = []
    
    for x, target in patterns:
        loss = net.train_step(x, target, lr=0.02)
        epoch_loss += loss
        
        # Get output for separation metric
        net.forward(x)
        outputs.append(net.output[0])  # First output dimension
    
    losses.append(epoch_loss / len(patterns))
    separations.append(abs(outputs[0] - outputs[1]))

# Plot training progress
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(losses, 'b-', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(separations, 'g-', linewidth=2)
ax.axhline(y=1.0, color='r', linestyle='--', label='Target separation')
ax.set_xlabel('Epoch')
ax.set_ylabel('Output Separation')
ax.set_title('Pattern Discrimination')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final test
print("\nFinal outputs:")
for i, (x, target) in enumerate(patterns):
    out = net.forward(x)
    print(f"  Pattern {i}: input={x}, target={target}, output={out.round(3)}")

print(f"\nFinal separation: {separations[-1]:.3f} (target: 1.0)")
print(f"Success: {separations[-1] > 0.5}")

---

## Summary

You've now seen the core algorithms behind Pulse Arithmetic Lab:

1. **Pulse counting** = addition in hardware
2. **Ternary weights** = neural networks without multiplication
3. **Spectral oscillators** = complex-valued dynamics with phase
4. **Equilibrium propagation** = learning without backpropagation

The ESP32-C6 firmware implements these in fixed-point arithmetic (Q15), running at:
- 1.1M pulses/second for counting
- 57k neuron-updates/second for inference
- 139 Hz for learning steps

### Next Steps

1. **Try the hardware**: Get an ESP32-C6 and flash the demos
2. **Read THEORY.md**: Deeper mathematical foundations
3. **Read CLAIMS.md**: Falsifiable scientific claims
4. **Contribute**: Help us improve at github.com/EntroMorphic/pulse-arithmetic-lab